# Implied Volatility

#### About the author

This notebook was adapted from this [post](http://blog.nag.com/2013/10/implied-volatility-using-pythons-pandas.html). The original author is Brian Spector from NAG.

### Introduction

In financial analysis, the famous [Black--Scholes formula](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model) prices an option as a function of six variables: Underlying Price, Interest Rate, Dividends, Strike Price, Time-to-Expiration, and Volatility. For a given option contract, we can observe the Underlying Price, Interest Rate, and Dividend Rate. In addition, the options' contract specifies the Strike Price and Time-to-Expiration.

The one variable we have to tweak is thus the volatility. Let us ask the question: What Volatility is such that the Black--Scholes formula price equals the market price?

$$
F(\text{Volatility}^*) = \text{Market Option Price},
$$

where $\text{Volatility}^*$ is the volatility **implied** by the market price or the [implied volatility](https://en.wikipedia.org/wiki/Implied_volatility). $F$ is a continuous function on $[0, 1]$ so we can use a root finder to get value(s) of $\text{Volatility}^*$ which solve(s) the above equation.

### NAG and Python libraries

We can use a few different NAG functions; [nag_zero_cont_func_brent](http://www.nag.com/numeric/CL/nagdoc_cl23/html/C05/c05ayc.html) finds the root using Brent's Algorithm, [nag_bsm_price](http://www.nag.com/numeric/CL/nagdoc_cl23/html/S/s30aac.html) calculates the theoretical option price, [nag_2d_cheb_fit_lines](http://www.nag.com/numeric/CL/nagdoc_cl23/html/E02/e02cac.html) performs a least squares Chebyshev fit to the volatility surface, and [nag_2d_cheb_eval](http://www.nag.com/numeric/CL/nagdoc_cl23/html/E02/e02cbc.html) evaluates the surface at intermediate points.

We will use the popular packages Python offers for numerical computing and data analysis: `numpy`, `scipy`, and `pandas`. The latter has fast and efficient data analysis tools to store and process large amounts of data. In addition, `pandas` comes with `numpy` and `ctypes` built into it, which allows easy integration with NAG's `nag4py` package. Visualizations will be made with `plotly`.

The code below runs on data from the Chicago Board of Options Exchange (CBOE) website. The CBOE provides options data in downloadable format [here](http://www.cboe.com/delayedquote/QuoteTableDownload.aspx). Make sure you download data during CBOE trading hours, so your graphs are not trivial.

Please get the `implied_volatility_utils` module (`implied_volatility_utils.py` file) from [here](http://www.nag.co.uk/blog_files/nag_imp_vol.zip).

In [1]:
# Load one of its functions to make sure you have it in the current working directory.
from implied_volatility_utils import generate_data

## Parse the data

In [2]:
import pandas as pd

In [3]:
pd.__version__

'0.16.2'

Take a look at the `QuoteData.dat` file; it starts with 2 lines of metadata, and looks like a CSV file.

In [4]:
quotedata = pd.read_csv('QuoteData.dat', header=2)

In [5]:
quotedata.head()

,Calls,Last Sale,Net,Bid,Ask,Vol,Open Int,Puts,Last Sale.1,Net.1,Bid.1,Ask.1,Vol.1,Open Int.1,Unnamed: 14
0,15 Jun 65.00 (AAPL1519F65),62.65,0,62.05,62.25,0,206,15 Jun 65.00 (AAPL1519R65),0.01,0,0.0,0.01,0,236,NaN
1,15 Jun 65.00 (AAPL1519F65-4),0.00,0,62.05,62.25,0,206,15 Jun 65.00 (AAPL1519R65-4),0.01,0,0.0,0.01,0,236,NaN
2,15 Jun 65.00 (AAPL1519F65-8),62.55,0,62.00,62.25,0,206,15 Jun 65.00 (AAPL1519R65-8),0.02,0,0.0,0.01,0,236,NaN
3,15 Jun 65.00 (AAPL1519F65-A),0.00,0,62.05,62.25,0,206,15 Jun 65.00 (AAPL1519R65-A),0.00,0,0.0,0.01,0,236,NaN
4,15 Jun 65.00 (AAPL1519F65-B),0.00,0,62.05,62.25,0,206,15 Jun 65.00 (AAPL1519R65-B),0.00,0,0.0,0.01,0,236,NaN


In [6]:
quotedata.describe()

,Last Sale,Net,Vol,Open Int,Last Sale.1,Net.1,Vol.1,Open Int.1,Unnamed: 14
count,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,7064.000000,0
mean,13.523344,-0.063621,39.737259,7779.500142,6.033988,0.020449,32.916195,5743.854332,NaN
std,20.289055,0.235779,421.766855,19202.093967,12.257830,0.167572,379.348601,11874.132766,NaN
min,0.000000,-5.310000,0.000000,0.000000,0.000000,-1.950000,0.000000,0.000000,NaN
25%,0.020000,0.000000,0.000000,165.000000,0.010000,0.000000,0.000000,105.000000,NaN
50%,2.900000,0.000000,0.000000,1014.000000,0.530000,0.000000,0.000000,1224.000000,NaN
75%,18.962500,0.000000,0.000000,5759.000000,5.350000,0.000000,0.000000,5428.000000,NaN
max,97.100000,3.300000,22162.000000,174284.000000,72.000000,4.560000,20699.000000,125734.000000,NaN


In [7]:
# Some filtering, clean-up.
quotedata = quotedata.fillna(0.0)
quotedata = quotedata[(quotedata['Last Sale'] > 0) | (quotedata['Last Sale.1'] > 0)]
quotedata = quotedata[quotedata['Calls'] > 0]
quotedata = quotedata.drop('Unnamed: 14', 1)

In [8]:
quotedata.describe()

,Last Sale,Net,Vol,Open Int,Last Sale.1,Net.1,Vol.1,Open Int.1
count,6334.000000,6334.000000,6334.000000,6334.000000,6334.000000,6334.000000,6334.000000,6334.000000
mean,15.081923,-0.070954,44.317019,8648.153300,6.729411,0.022805,36.709820,6380.140354
std,20.870691,0.247951,445.184473,20091.901379,12.762977,0.176814,400.442175,12380.346472
min,0.000000,-5.310000,0.000000,0.000000,0.000000,-1.950000,0.000000,0.000000
25%,0.150000,0.000000,0.000000,273.000000,0.040000,0.000000,0.000000,269.000000
50%,4.825000,0.000000,0.000000,1373.000000,0.920000,0.000000,0.000000,1598.000000
75%,22.957500,0.000000,0.000000,7013.000000,6.400000,0.000000,0.000000,6368.000000
max,97.100000,3.300000,22162.000000,174284.000000,72.000000,4.560000,20699.000000,125734.000000


Dimensions can be accessed in two possible ways:

In [9]:
quotedata.Calls  # or quotedata['Calls']

0           15 Jun 65.00 (AAPL1519F65)
1         15 Jun 65.00 (AAPL1519F65-4)
2         15 Jun 65.00 (AAPL1519F65-8)
5         15 Jun 65.00 (AAPL1519F65-E)
6         15 Jun 65.00 (AAPL1519F65-I)
7         15 Jun 65.00 (AAPL1519F65-J)
8         15 Jun 65.00 (AAPL1519F65-O)
9         15 Jun 65.00 (AAPL1519F65-P)
11        15 Jun 65.00 (AAPL1519F65-X)
12        15 Jun 65.00 (AAPL1519F65-Y)
13          15 Jun 70.00 (AAPL1519F70)
14        15 Jun 70.00 (AAPL1519F70-4)
15        15 Jun 70.00 (AAPL1519F70-8)
17        15 Jun 70.00 (AAPL1519F70-B)
18        15 Jun 70.00 (AAPL1519F70-E)
19        15 Jun 70.00 (AAPL1519F70-I)
20        15 Jun 70.00 (AAPL1519F70-J)
21        15 Jun 70.00 (AAPL1519F70-O)
22        15 Jun 70.00 (AAPL1519F70-P)
24        15 Jun 70.00 (AAPL1519F70-X)
25        15 Jun 70.00 (AAPL1519F70-Y)
26          15 Jun 75.00 (AAPL1519F75)
27        15 Jun 75.00 (AAPL1519F75-4)
28        15 Jun 75.00 (AAPL1519F75-8)
29        15 Jun 75.00 (AAPL1519F75-A)
31        15 Jun 75.00 (A

Let us read the metadata now (we could have started with this).

In [10]:
qd = open('QuoteData.dat', 'r')

In [11]:
qd_head = []
qd_head.append(qd.readline())
qd_head.append(qd.readline())
qd.close()

In [12]:
first_line = qd_head[0].split(',')

In [13]:
second_line = qd_head[1].split()

In [14]:
underlyingprice = float(first_line[1])

In [15]:
# Set to hold expiration dates
cumulative_month = {'Jan': 31, 'Feb': 57, 'Mar': 90,
                    'Apr': 120, 'May': 151, 'Jun': 181,
                    'Jul': 212, 'Aug': 243, 'Sep': 273,
                    'Oct': 304, 'Nov': 334, 'Dec': 365}

month, day = second_line[:2]
today = cumulative_month[month] + int(day) - 30
current_year = int(second_line[2])

### Get the Options' Expiration Date

In [16]:
def get_expiration(x):
    monthday = x.split()
    adate = monthday[0] + ' ' + monthday[1]
    return (int(monthday[0]) - (current_year % 2000)) * 365 + cumulative_month[monthday[1]]

In [17]:
expiration = quotedata['Calls'].apply(get_expiration)
expiration.name = 'Expiration'

### Get the Strike Prices

In [18]:
def get_strike(x):
    monthday = x.split()
    return float(monthday[2])

In [19]:
strike = quotedata['Calls'].apply(get_strike)
strike.name = 'Strike'

In [20]:
quotedata = quotedata.join(expiration).join(strike)

Take a look at a piece of data.

In [21]:
quotedata.ix[1000, :]

Calls          15 Jun 121.00 (AAPL1526F121-Y)
Last Sale                               10.82
Net                                         0
Bid                                      6.30
Ask                                      6.40
Vol                                         0
Open Int                                  434
Puts           15 Jun 121.00 (AAPL1526R121-Y)
Last Sale.1                              0.11
Net.1                                       0
Bid.1                                    0.10
Ask.1                                    0.12
Vol.1                                       0
Open Int.1                               3200
Expiration                                181
Strike                                    121
Name: 1000, dtype: object

## Calculate Implied Volatility of Calls

Let us denote the underlying price by S, the interest rate by r, and the dividend rate by q.

In [22]:
S = 100.0
r = 0.025
q = 0.025

We will consider a small sample of our data to keep computing times under a few minutes.

In [81]:
n = 200  # Feel free to increase!

In [82]:
sampledata = quotedata.sample(n)

In [83]:
sampledata.head()

,Calls,Last Sale,Net,Bid,Ask,Vol,Open Int,Puts,Last Sale.1,Net.1,Bid.1,Ask.1,Vol.1,Open Int.1,Expiration,Strike
2870,15 Jul 175.00 (AAPL1517G175-O),0.01,0.00,0.0,0.01,0,5958,15 Jul 175.00 (AAPL1517S175-O),49.25,0.00,46.90,48.30,0,1019,212,175.00
11,15 Jun 65.00 (AAPL1519F65-X),62.05,0.00,62.05,62.25,0,206,15 Jun 65.00 (AAPL1519R65-X),0.04,0.00,0.0,0.01,0,236,181,65.00
5918,16 Jan 125.71 (AAPL1615A125.71-4),9.86,-0.14,9.60,9.70,1,26779,16 Jan 125.71 (AAPL1615M125.71-4),8.25,0.00,8.50,8.60,0,15160,396,125.71
5569,16 Jan 102.14 (AAPL1615A102.14-X),29.71,0.00,26.30,26.50,0,2630,16 Jan 102.14 (AAPL1615M102.14-X),1.59,0.00,1.63,1.67,0,4717,396,102.14
873,15 Jun 112.00 (AAPL1526F112-8),0.00,0.00,15.20,15.35,0,176,15 Jun 112.00 (AAPL1526R112-8),0.05,-0.01,0.04,0.06,5,1225,181,112.00


In [58]:
sampledata.head()

,Calls,Last Sale,Net,Bid,Ask,Vol,Open Int,Puts,Last Sale.1,Net.1,Bid.1,Ask.1,Vol.1,Open Int.1,Expiration,Strike
6146,16 Jan 165.00 (AAPL1615A165-Y),0.70,0.0,0.62,0.66,0,7855,16 Jan 165.00 (AAPL1615M165-Y),0.00,0,38.80,39.10,0,663,396,165
1255,15 Jun 141.00 (AAPL1526F141-J),0.05,0.0,0.0,0.01,0,1149,15 Jun 141.00 (AAPL1526R141-J),0.00,0,13.70,13.85,0,20,181,141
5877,16 Jan 120.00 (AAPL1615A120-P),12.90,-0.4,12.80,12.95,9,93154,16 Jan 120.00 (AAPL1615M120-P),5.85,0,6.00,6.10,0,50689,396,120
6165,16 Jan 170.00 (AAPL71615A170),0.61,0.0,,,0,310,16 Jan 170.00 (AAPL71615M170),0.00,0,,,0,0,396,170
2519,15 Jul 85.00 (AAPL1517G85-S),42.95,0.0,41.85,43.75,0,924,15 Jul 85.00 (AAPL1517S85-S),0.01,0,0.0,0.02,0,26381,212,85


Let us generate option prices as given by the Black--Scholes formula, using test volatility curves. These functions were written for `numpy` objects, not `pandas` objects, hence the conversions on our data structures.

In [84]:
option_prices = generate_data(sampledata.Strike.as_matrix(), S, sampledata.Expiration.as_matrix(), r, q)

In [26]:
option_prices.keys()

['put', 'call']

We will compare computing times when using solvers from `scipy` vs NAG bindings for Python.

In [27]:
import time

### SciPy method

In [105]:
from implied_volatility_utils import scipy_calcvol as calcvol

In [106]:
start_time = time.clock()
imp_vol = calcvol(sampledata.Strike.as_matrix(), S, sampledata.Expiration.tolist(), r, q, option_prices)
elapsed_time = time.clock() - start_time

In [107]:
elapsed_time

758.209914

### Nag4Py method

In [31]:
from implied_volatility_utils import (nag4py_package_check,
                                      nag4py_calcvol as calcvol)
nag4py_package_check()

In [85]:
start_time = time.clock()
imp_vol = calcvol(sampledata.Strike.as_matrix(), S, sampledata.Expiration.tolist(), r, q, option_prices)
elapsed_time = time.clock() - start_time

In [86]:
elapsed_time

46.92882899999998

The computing time is faster by an order of magnitude.

## Visualize!

In [34]:
import plotly.plotly as py
import plotly.graph_objs as pgo

In [35]:
imp_vol.keys()

['put', 'call']

In [87]:
data = pgo.Data([
        pgo.Scatter(
            x=sampledata.Strike,
            y=imp_vol['call'],
            name='call',
            mode='markers',
            marker=pgo.Marker(color='red')
        ),
        pgo.Scatter(
            x=sampledata.Strike,
            y=imp_vol['put'],
            name='put',
            mode='markers',
            marker=pgo.Marker(color='blue')
        )
])

In [88]:
layout = pgo.Layout(
    title='Implied Volatility Curve',
    xaxis=pgo.XAxis(title='Strike Price'),
    yaxis=pgo.YAxis(title='Implied Volatility'),
    hovermode='closest'
)

In [89]:
fig = pgo.Figure(data=data, layout=layout)

In [90]:
py.iplot(fig, filename='implied-volatility-nag4py-2d')

The draw time for this plot will be slow for all clients.


/usr/local/lib/python2.7/dist-packages/plotly/plotly/plotly.py:1261: UserWarning:

Estimated Draw Time Too Long



Let us plot implied volatility vs both Strike and Expiration.

First, we need to massage the `imp_vol` dictionary into an array so we can do a surface plot.

In [91]:
df = pd.DataFrame({'strike': sampledata.Strike.tolist() * n,
                   'expiration': sampledata.Expiration.repeat(n),
                   'call': imp_vol['call']})

In [92]:
df.head()

,call,expiration,strike
2870,0.412557,212,175.00
2870,0.324475,212,65.00
2870,0.313221,212,125.71
2870,0.300091,212,102.14
2870,0.302880,212,112.00


In [93]:
df = df.drop_duplicates(['expiration', 'strike'])

In [94]:
pivot_df = df.reset_index().pivot('expiration', 'strike', 'call')

In [95]:
pivot_df

strike,34.29,37.14,40.0,44.29,51.43,54.29,57.14,58.57,60.0,65.0,...,148.0,150.0,155.0,160.0,165.0,170.0,175.0,180.0,185.0,190.0
expiration,,,,,,,,,,,,,,,,,,,,,
181,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038440,0.042907,0.046567,0.056202,...,0.110959,0.111707,0.113512,0.115224,0.116841,0.118378,0.119820,0.121183,0.122474,0.123700
212,0.386388,0.378836,0.371948,0.362095,0.347272,0.341787,0.336734,0.334321,0.331989,0.324475,...,0.346080,0.350045,0.360508,0.372000,0.384520,0.397952,0.412557,0.427996,0.444557,0.462278
243,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
546,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
761,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [96]:
trace = pgo.Surface(
    x=sampledata.Strike,
    y=sampledata.Expiration,
    z=pivot_df.as_matrix()
)

In [97]:
data1 = pgo.Data([trace])

In [101]:
pivot_df.to_csv('call.csv')

In [98]:
layout1 = pgo.Layout(
    title='Implied Volatility',
    scene=pgo.Scene(
        xaxis=pgo.XAxis(title='Strike Price'),
        yaxis=pgo.YAxis(title='Time to Expiration')
    )
)

In [99]:
fig1 = pgo.Figure(data=data1, layout=layout1)

In [100]:
py.iplot(fig1, filename='implied-volatility-nag4py-3d')